In [44]:
import pandas as pd
from pulp import *
import re

In [45]:
data = pd.read_csv("./data/IPL-aggregated-stats_final.csv")

In [46]:
data.head()

,PLAYER,Pts_sum,Pts_mean,years_played,Mat_sum,Mat_mean,Wkts_sum,Wkts_mean,Dots_sum,Dots_mean,...,6s_mean,Catches_sum,Catches_mean,Stumpings_sum,Stumpings_mean,position,overseas_or_indian,median_season_ratings,avg_season_ratings,base_price
0,ABdeVilliers,1712.5,155.681818,11,139,12.636364,0,0.000000,0,0.000000,...,18.181818,80,7.272727,7,0.636364,BAT,Overseas,50.344828,51.410021,1.50
1,AaronFinch,751.0,83.444444,9,75,8.333333,1,0.111111,13,1.444444,...,7.444444,24,2.666667,0,0.000000,BAT,Overseas,11.699010,27.457438,0.75
2,AbhishekJhunjhunwala,120.5,40.166667,3,21,7.000000,1,0.333333,27,9.000000,...,1.666667,10,3.333333,0,0.000000,BAT,Indian,2.472464,4.623134,0.30
3,AbhishekNayar,340.5,48.642857,7,60,8.571429,9,1.285714,69,9.857143,...,2.857143,13,1.857143,0,0.000000,BAT,Indian,2.480645,8.571047,0.30
4,AdamGilchrist,1091.5,181.916667,6,80,13.333333,1,0.166667,1,0.166667,...,15.333333,51,8.500000,16,2.666667,WK,Overseas,70.380747,61.764019,2.00


In [47]:
data.overseas_or_indian.value_counts()

Indian      135
Overseas    104
Name: overseas_or_indian, dtype: int64

### The objective function along with other constraints are mentioned below

$$ max \sum_{players =1}^N rating$$
s.t $$ min-batsman <  chosen-batsman < max-batsman $$
s.t $$ min-batsman <  chosen-batsman < max-batsman $$
s.t $$ min-bowlers <  chosen-bowlers < max-bowlers $$
s.t $$ min-allrounders <  chosen-all-rounders < max-allrounders $$
s.t $$ wicketkeepers = 1 $$

### Creating the set of players that will go into the auction

In [48]:
players = list(data["PLAYER"])
indians = list(data[data["overseas_or_indian"]=="Indian"]["PLAYER"])
overseas = list(data[data["overseas_or_indian"]=="Overseas"]["PLAYER"])
batsman = list(data[data.position=="BAT"]["PLAYER"])
bowlers = list(data[data.position=="BWL"]["PLAYER"])
allrounders = list(data[data.position=="AR"]["PLAYER"])
wks = list(data[data.position=="WK"]["PLAYER"])

In [49]:
median_ratings = dict(zip(players, list(data["median_season_ratings"])))
base_prices = dict(zip(players, list(data["base_price"])))

### Lets implement a Binary Integer Programming strategy

In [52]:
players_lp = LpVariable.dicts("Chosen",players,0,1,cat='Integer')

In [53]:
def filter_lp_dicts(players_lp, criteria_list):
    filter_dict = {}
    for player in criteria_list:
        filter_dict[player] = players_lp[player]
    return filter_dict

In [54]:
batsman_lp = filter_lp_dicts(players_lp, batsman)
bowlers_lp = filter_lp_dicts(players_lp, bowlers)
ar_lp = filter_lp_dicts(players_lp, allrounders)
wk_lp = filter_lp_dicts(players_lp, wks)
indians_lp = filter_lp_dicts(players_lp, indians)
overseas_lp = filter_lp_dicts(players_lp, overseas)

In [55]:
def constructLpSumArr(dict_lp):
    lp_arr = []
    for k,v in dict_lp.items():
        lp_arr += lpSum(dict_lp[k])
    return lp_arr

In [56]:
rewards = []
costs = []
MAX_AVAILABLE_COST = 30
for k, v in players_lp.items():
    costs += lpSum(base_prices[k] * players_lp[k])
    rewards += lpSum(median_ratings[k] * players_lp[k])
prob+= lpSum(rewards)
prob += lpSum(costs) <= MAX_AVAILABLE_COST
prob += lpSum(constructLpSumArr(wk_lp)) == 1
prob += lpSum(constructLpSumArr(players_lp)) == 11
prob += lpSum(constructLpSumArr(batsman_lp)) >= 3
prob += lpSum(constructLpSumArr(batsman_lp)) <= 5
prob += lpSum(constructLpSumArr(bowlers_lp)) >= 3
prob += lpSum(constructLpSumArr(bowlers_lp))<= 5
prob += lpSum(constructLpSumArr(ar_lp)) >= 1
prob += lpSum(constructLpSumArr(ar_lp)) <= 3
prob += lpSum(constructLpSumArr(indians_lp)) == 7
prob += lpSum(constructLpSumArr(overseas_lp)) == 4

In [57]:
prob.solve()

1

In [58]:
prob

IPL_Auction:
MAXIMIZE
50.3448275862069*Chosen_ABdeVilliers + 11.6990099009901*Chosen_AaronFinch + 2.4724637681159423*Chosen_AbhishekJhunjhunwala + 2.4806451612903233*Chosen_AbhishekNayar + 70.3807471264368*Chosen_AdamGilchrist + 4.077748312439732*Chosen_AjanthaMendis + 50.23255813953488*Chosen_AjinkyaRahane + 11.919799139167864*Chosen_AjitAgarkar + 2.467512274959084*Chosen_AkshdeepNath + 62.85454545454546*Chosen_AlbieMorkel + 38.155011655011656*Chosen_AmbatiRayudu + 50.21739130434783*Chosen_AmitMishra + 69.0*Chosen_AndreRussell + 21.57246376811594*Chosen_AndrewSymonds + 32.149253731343286*Chosen_AndrewTye + 16.04479166666667*Chosen_AngeloMathews + 81.0*Chosen_AnilKumble + 8.895238095238096*Chosen_AnkitRajpoot + 2.4800000000000004*Chosen_AnkitSharma + 2.4740740740740743*Chosen_AnureetSingh + 28.163912579957355*Chosen_AshishNehra + 5.706684825785951*Chosen_AshishReddy + 21.542386671832624*Chosen_AshokDinda + 62.71250000000001*Chosen_AxarPatel + 67.5*Chosen_AzharMahmood + 2.47092198581560

In [59]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [60]:
summary(prob)

Variables:

Chosen_AnilKumble = 1.0
Chosen_BhuvneshwarKumar = 1.0
Chosen_ChrisGayle = 1.0
Chosen_DavidWarner = 1.0
Chosen_HarbhajanSingh = 1.0
Chosen_JacquesKallis = 1.0
Chosen_MunafPatel = 1.0
Chosen_RashidKhan = 1.0
Chosen_RishabhPant = 1.0
Chosen_SureshRaina = 1.0
Chosen_VirenderSehwag = 1.0
---------------------------------------

Constraints:
2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 = 22.0
---------------------------------------

Score:
81.0*1.0 + 80.66666666666667*1.0 + 80.47619047619048*1.0 + 80.55555555555556*1.0 + 80.49107142857143*1.0 + 70.33333333333333*1.0 + 80.38461538461539*1.0 + 81.66666666666667*1.0 + 75.96428571428572*1.0 + 80.51315789473685*1.0 + 80.58823529411767*1.0 = 872.6397784147397


### Obviously players wont get sold at their base price. We need to use some inflated price to predict their final sell price.

In [86]:
def get_bidding_increment(price):
    if price <= 0.75:
        return 0.1
    elif (price >= 0.75 ) and (price < 1.5):
        return 0.2
    elif (price >= 1.5) and (price < 3.5):
        return 0.3
    elif (price >= 3.5) and (price < 5):
        return 0.5
    else:
        return 1

In [89]:
def price_after_betting(base_price, rounds):
    price = base_price
    while(rounds > 0):
        price += get_bidding_increment(price)
        price = round(price, 1)
        rounds-=1
    return price

### We say all players atleast go for 5 rounds of betting (5 times the price was increased)

In [106]:
inflated_price = dict(zip(players, list(data["base_price"].apply(lambda x : price_after_betting(x, 3)))))

In [107]:
prob = LpProblem("IPL_Auction", LpMaximize)
rewards = []
costs = []
MAX_AVAILABLE_COST = 30
for k, v in players_lp.items():
    costs += lpSum(inflated_price[k] * players_lp[k])
    rewards += lpSum(median_ratings[k] * players_lp[k])
prob+= lpSum(rewards)
prob += lpSum(costs) <= MAX_AVAILABLE_COST
prob += lpSum(constructLpSumArr(wk_lp)) == 1
prob += lpSum(constructLpSumArr(players_lp)) == 11
prob += lpSum(constructLpSumArr(batsman_lp)) >= 3
prob += lpSum(constructLpSumArr(batsman_lp)) <= 5
prob += lpSum(constructLpSumArr(bowlers_lp)) >= 3
prob += lpSum(constructLpSumArr(bowlers_lp))<= 5
prob += lpSum(constructLpSumArr(ar_lp)) >= 1
prob += lpSum(constructLpSumArr(ar_lp)) <= 3
prob += lpSum(constructLpSumArr(indians_lp)) == 7
prob += lpSum(constructLpSumArr(overseas_lp)) == 4

In [108]:
prob.solve()

1

In [109]:
summary(prob)

Variables:

Chosen_AjinkyaRahane = 1.0
Chosen_AnilKumble = 1.0
Chosen_AxarPatel = 1.0
Chosen_BhuvneshwarKumar = 1.0
Chosen_DaleSteyn = 1.0
Chosen_DavidWarner = 1.0
Chosen_LasithMalinga = 1.0
Chosen_RashidKhan = 1.0
Chosen_RishabhPant = 1.0
Chosen_SureshRaina = 1.0
Chosen_VirenderSehwag = 1.0
---------------------------------------

Constraints:
1.6*1.0 + 2.9*1.0 + 2.4*1.0 + 2.9*1.0 + 2.4*1.0 + 2.9*1.0 + 2.9*1.0 + 2.9*1.0 + 2.9*1.0 + 2.9*1.0 + 2.9*1.0 = 29.599999999999994
---------------------------------------

Score:
50.23255813953488*1.0 + 81.0*1.0 + 62.71250000000001*1.0 + 80.66666666666667*1.0 + 60.44563279857398*1.0 + 80.55555555555556*1.0 + 80.52631578947368*1.0 + 81.66666666666667*1.0 + 75.96428571428572*1.0 + 80.51315789473685*1.0 + 80.58823529411767*1.0 = 814.8715745196117
